1. [20 pts] In this assignment, we will approach the problem with sentiment analysis in our pipeline to classify movie reviews.
Use the Opinion Lexicon from the source: http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
Download the two files, positive-words.txt and negative-words.txt and use them to opinion-score each pooled review (i.e., ignore the original class labels). One approach can be counting the number of occurrences of the lexicon words and scoring appropriately.
(Hint: Quantize the positive and negative frequencies to 0 and 1,
i.e., predict = (opinion_score[:,0] > opinion_score[:,1]) + 0
)
What is the prediction accuracy when done like in the above?

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 19.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.0 MB/s eta 0:00:00eta 0:00:01


2. [20 pts] Use only the first 1000 words in Tf-Idf features and compute 10-fold CV classification evaluation performance. We expect a lower score compared to when using the full set of features. Recall that when Tf-Idf features are ranked, the top features exhibit a high information content, since Idf normalizes the document frequencies properly.

3. [20 pts] Add the two columns of positive and negative opinions (e.g., counts) to the 1000 Tf-Idf features in (2.) and re-compute the 10-fold CV classification evaluation. Did opinions from (1.) help?

4. [20 pts] Compare the ranked features as in the previous homework, indicate where the two added opinion features located with respect to the high-ranked Tf-Idf features (as we conducted in the previous assignments)?

5. [20 pts] We were unable to improve the labeling and/or classification as it looks like the IMDB review dataset ground truth is generated from something else.
List some possibilities about the source of the ground truth of original dataset.
Now attempt to extract the voting score from the review (if exists), for example the first review has it.
Outline an approach.